### Топ-100 самых частовстречаемых слов в книге "Война и Мир"

Откроем файл и прочитаем весь текст книги

In [1]:
with open('Lev Tolstoy - Voyna i mir.txt') as f:
    dirty_text = f.read()

Разобьём его на предложения

In [2]:
from rusenttokenize import ru_sent_tokenize

dirty_sentences = ru_sent_tokenize(dirty_text)
print(len(dirty_sentences))

15283


Очистим от знаков препинания

In [3]:
import string

def clear(s):
    for ch in string.punctuation:
        s = s.replace(ch,'')
    return s.lower()

clean_sentences = [clear(sentence) for sentence in dirty_sentences]

Сформируем список стоп-слов. Т.к. в книге часто встречаются французский, то надо учесть и русские, и французские слова.

In [4]:
from nltk.corpus import stopwords
sw = stopwords.words('russian') + stopwords.words('french') + ['a', '–'] # русские, французские и выпавшие из обоих списков 
                                                                         # объекты

Выкинем стоп слова, остальные нормализуем. Нормализация выполняется только для русского языка, т.к. несмотря на то, что во французском языке тоже есть падежи, они формируются при помощи предлогов, а т.к. они находятся среди стоп-слов то нормализация им не нужна.

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def approved_by_morph(word, morph):
    p = morph.parse(word)
    # отбойник на случай, если морфологический анализ не смог выдать результата
    if(p == None) or (len(p) == 0):
        return True
    
    if('NOUN' in p[0].tag) or ('LATN' in p[0].tag):   # отбираем либо существительные, либо то, 
                                                      # что морфологический анализ посчитал иностранным словом
        return True
    else:
        return False
        
def normalize(word, morph):
    p = morph.parse(word)
    if(p == None) or (len(p) == 0):
        return word
    return p[0].normal_form

def post_clear(words, forbidden_words, morph):
    return [normalize(word, morph) for word in words if word not in forbidden_words and approved_by_morph(word, morph)]

prepared_text = [post_clear(s.split(), sw, morph) for s in clean_sentences]

Сформируем граф, отсортируем слова по степени связности с другими

In [6]:
graph = {}

for sentence in prepared_text:
    for word in sentence:
        graph[word] = {}

for sentence in prepared_text:
    for word1 in sentence:
        for word2 in sentence:
            #if (word1 < word2):
                #word1, word2 = word2, word1 # swap values
            if (word1 == word2):
                continue
            if word2 in graph[word1]:
                graph[word1][word2] += 1
            else:
                graph[word1][word2] = 1
                
            if word1 in graph[word2]:
                graph[word2][word1] += 1
            else:
                graph[word2][word1] = 1

vertices_count = len(graph)
edge_count = 0
top = {}
for edge in graph:
    top[edge] = 0
    for word2 in graph[edge]:
        graph[edge][word2] /= 2
        edge_count += 1
        top[edge] += 1

top = dict(sorted(top.items(), key = lambda item: -item[1]))
        
print(vertices_count)
print(edge_count)

7847
273766


Посмотрим на наш "топ"

In [10]:
top100 = []
keys = list(top.keys())
for i in range(0, 100):
    top100.append(keys[i])
    
print(top100)

['князь', 'человек', 'пьер', 'андрей', 'лицо', 'рука', 'время', 'дело', 'глаз', 'ростов', 'граф', 'слово', 'наташа', 'княжна', 'голос', 'день', 'анна', 'дом', 'друг', 'минута', 'голова', 'марья', 'улыбка', 'офицер', 'государь', 'место', 'борис', 'сторона', 'николай', 'жизнь', 'отец', 'комната', 'год', 'лошадь', 'солдат', 'нога', 'разговор', 'сила', 'сын', 'вид', 'графиня', 'чувство', 'император', 'c’est', 'денисов', 'генерал', 'павлович', 'кутузов', 'plus', 'свет', 'москва', 'армия', 'княгиня', 'выражение', 'дверь', 'стол', 'письмо', 'шаг', 'мысль', 'comme', 'вечер', 'войско', 'звук', 'василий', 'жена', 'бог', 'француз', 'соня', 'часть', 'любовь', 'наполеон', 'душа', 'гость', 'дорога', 'общество', 'брат', 'петербург', 'анатоль', 'обед', 'безухов', 'женщина', 'мать', 'михаилович', 'тысяча', 'si', 'движение', 'бонапарт', 'сражение', 'адъютант', 'гора', 'счастие', 'положение', 'взгляд', 'багратион', 'война', 'изз', 'платье', 'деревня', 'командир', 'час']


Всё готово к экспорту в граф

In [15]:
with open('graph_nodes.csv', 'w') as f:
    f.write('Id;Label\n')
    for word in top100:
        f.write(word + ';' + word + '\n')

In [24]:
with open('graph_edges.csv', 'w') as f:
    f.write('Source;Target;Weight\n')
    for word in top100:
        for edge in graph[word]:
            # граф получается слишком плотным, поэтому выкидываем все рёбра весом меньше 10
            if (edge not in top100) or (graph[word][edge] <= 10.0):
                continue
            f.write(word + ';' + edge + ';' + str(graph[word][edge]) + '\n')

Импортируем файлы в Gephi, настраиваем отображение, смотрим на результат

![](graph2.png)